### Absenteeism as Work Analysis

Notebook 3/3 

At this point, we have created a model that can predict absenteeism for any new individuals, and also a module that will take care of the preprocessing of our data.  

Now we can take a look at predicting new data and even the entire dataset. 

Starting off by importing the absenteeism_module that we created in the previous notebook.  
We will also import pymysql to connect to a database and config to import the credentials to the database. 
This import will also import the libraries used in the module (pickle, numpy, pandas, and sklearn). 

In [1]:
from absenteeism_module import *
import pymysql
import config

In [2]:
df = pd.read_csv('absenteeism_new_data.csv', delimiter=',')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       40 non-null     int64  
 1   Reason for Absence       40 non-null     int64  
 2   Date                     40 non-null     object 
 3   Transportation Expense   40 non-null     int64  
 4   Distance to Work         40 non-null     int64  
 5   Age                      40 non-null     int64  
 6   Daily Work Load Average  40 non-null     float64
 7   Body Mass Index          40 non-null     int64  
 8   Education                40 non-null     int64  
 9   Children                 40 non-null     int64  
 10  Pets                     40 non-null     int64  
dtypes: float64(1), int64(9), object(1)
memory usage: 3.6+ KB


We can initialize our model with the mode and scaler that we created in the previous notebook.

In [4]:
model = AbsenteeismModel('model.pkl', 'scaler.pkl')

Then we can preprocess our data. 

In [5]:
model.load_and_clean_data('absenteeism_new_data.csv')

Once our data is clean, we can use the model to predict absenteeism for new individuals.

In [6]:
df_new_obs = model.predicted_outputs()

In [7]:
df_new_obs.head(3)

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Probability,Prediction
0,0,0,0,1,6,4,179,30,19,1,0,0,0.115844,0
1,1,0,0,0,6,0,361,28,27,0,1,4,0.874164,1
2,0,0,0,1,6,2,155,34,25,0,2,0,0.269384,0


And just like that, we are successfully able to predict absenteeism for new individuals.

We can combine this new dataset with the original dataset that we used to train our model while also predicting absenteeism for the original dataset as well. 

In [8]:
model_old_data = AbsenteeismModel('model.pkl', 'scaler.pkl')
model_old_data.load_and_clean_data('Absenteeism-data.csv')
df_old_obs = model_old_data.predicted_outputs()

In [9]:
all_data = pd.concat([df_new_obs, df_old_obs], axis=0).reset_index(drop=True)

In [10]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Reason_1                740 non-null    int32  
 1   Reason_2                740 non-null    int32  
 2   Reason_3                740 non-null    int32  
 3   Reason_4                740 non-null    int32  
 4   Month Value             740 non-null    int64  
 5   Day of the Week         740 non-null    int64  
 6   Transportation Expense  740 non-null    int64  
 7   Age                     740 non-null    int64  
 8   Body Mass Index         740 non-null    int64  
 9   Education               740 non-null    int64  
 10  Children                740 non-null    int64  
 11  Pets                    740 non-null    int64  
 12  Probability             740 non-null    float64
 13  Prediction              740 non-null    int64  
dtypes: float64(1), int32(4), int64(9)
memory u

In [11]:
all_data.head(3)

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Probability,Prediction
0,0,0,0,1,6,4,179,30,19,1,0,0,0.115844,0
1,1,0,0,0,6,0,361,28,27,0,1,4,0.874164,1
2,0,0,0,1,6,2,155,34,25,0,2,0,0.269384,0


We can now import this data into a MySQL database.  
Note that setting up the proper database and table in MySQL is done outside of this notebook but can be found in the 'sql_script.sql' file.
Set up proper database and table in MySQL prior to continuing.

Start of by creating a connection to the database. Note that the password is obfuscated in this notebook. 

In [12]:
conn = pymysql.connect(user=config.sql_username, password=config.sql_password, database='predicted_outputs')

Then we initialize our cursor.

In [13]:
cursor = conn.cursor()

In [14]:
cursor.execute('SELECT * FROM predicted_outputs')

0

We can see that there is currently no data in the predicted_outputs table. 

In [15]:
all_data.shape

(740, 14)

In [16]:
all_data.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Day of the Week', 'Transportation Expense', 'Age',
       'Body Mass Index', 'Education', 'Children', 'Pets', 'Probability',
       'Prediction'], dtype=object)

In [17]:
all_data.columns.values[7]

'Age'

We can access the individual values with the method shown below. 

In [18]:
print(
    all_data[all_data.columns.values[7]][0], 
    all_data[all_data.columns.values[7]][1]
)

30 28


With that, we can use a nested for loop to insert all of the values into the database. We will first create a string with our insert query. 

In [19]:
insert_query = 'INSERT INTO predicted_outputs VALUES'

for i in range(all_data.shape[0]):
    insert_query += '('
    
    for j in range(all_data.shape[1]):
        insert_query += str(all_data[all_data.columns.values[j]][i]) + ', '
    
    insert_query = insert_query[:-2] + '), '

insert_query = insert_query[:-2] + ';'

In [20]:
insert_query[:300]

'INSERT INTO predicted_outputs VALUES(0, 0, 0, 1, 6, 4, 179, 30, 19, 1, 0, 0, 0.11584410710059391, 0), (1, 0, 0, 0, 6, 0, 361, 28, 27, 0, 1, 4, 0.8741642059991989, 1), (0, 0, 0, 1, 6, 2, 155, 34, 25, 0, 2, 0, 0.2693838420603541, 0), (0, 0, 0, 1, 6, 4, 179, 40, 22, 1, 2, 0, 0.18975080397744465, 0), (1'

We can see that our insert query was correctly created.

We can now execute our insert query to our data to the database.

In [21]:
cursor.execute(insert_query)

740

In [22]:
cursor.execute('SELECT * FROM predicted_outputs')

740

We can see that there our data was successfully inserted into the database.

In [23]:
conn.commit()

In [24]:
conn.close()

With that, we can save our predicted dataframe to a CSV file and conduct further analysis and create visualiztions using Tableau. 

In [25]:
all_data.to_csv('absenteeism_predictions.csv', index=False)

In [26]:
pd.read_csv('absenteeism_predictions.csv').head(3)

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Probability,Prediction
0,0,0,0,1,6,4,179,30,19,1,0,0,0.115844,0
1,1,0,0,0,6,0,361,28,27,0,1,4,0.874164,1
2,0,0,0,1,6,2,155,34,25,0,2,0,0.269384,0
